# **Data Collection - Fetch Dataset from Kaggle**

## Objectives
* Fetch data from Kaggle and save as raw data to prepare it for further processes.

## Inputs
* Kaggle JSON file - the authentication token 

## Outputs
* Generate Dataset: inputs/mildew_dataset/cherry-leaves 

## Additional Comments
* No additional comments



## Import pagkages

In [1]:
! pip install -r /workspaces/mildew-detection-in-cherry-leaves/requirements.txt

uirement already satisfied: h5py==3.1.0 in /home/codeany/.pyenv/versions/3.8.12/lib/python3.8/site-packages (from -r /workspaces/mildew-detection-in-cherry-leaves/requirements.txt (line 18)) (3.1.0)


In [2]:
import numpy
import os

## Change Working Directory

* To change the working directory from its current folder to its parent folder
* To access the current directory with os.getcwd()

In [3]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/mildew-detection-in-cherry-leaves/jupyter_notebooks'

To make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


To confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'/workspaces/mildew-detection-in-cherry-leaves'

# Install Kaggle

In [6]:
# install kaggle package
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


---

Run the cell below to change the kaggle configuration directory to current working directory and permission of kaggle authentication JSON

In [7]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Get the dataset path from Kaggle url.  When you are viewing the dataset at Kaggle, check what is after https://www.kaggle.com/ (in some case kaggle.com/datasets). You should copy that at KaggleDatasetPath.

* Set the destination folder
* Set the Kaggle Dataset and Download it

In [8]:
# Set the destination folder to download Kaggle Dataset
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/mildew_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 91%|██████████████████████████████████▌   | 50.0M/55.0M [00:00<00:00, 57.0MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:00<00:00, 70.4MB/s]


* Unzip the downloaded file, and delete the zip file.

In [9]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Preparation

### Check and remove non-image files

In [10]:
# Function to remove non-image files
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir) 
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # remove non-image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))

In [11]:
# Remove non-image files
remove_non_image_file(my_data_dir='inputs/mildew_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


### Split train, validation, and test sets

In [12]:
import os
import shutil
import random
import joblib

my_data_dir="inputs/mildew_dataset/cherry-leaves"

# Function to split train, validation, and test images
def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  # gets classes labels
  labels = os.listdir(my_data_dir) # To get only the folder name
  if 'test' in labels:
    pass
  else: 
    # create train, validation, and test folders with classess labels sub-folder
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          # move given file to train set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          # move given file to validation set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          # move given file to test set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)

In [13]:
# Split train, validation, and test images
split_train_validation_test_images(my_data_dir=f"inputs/mildew_dataset/cherry-leaves",
                        train_set_ratio = 0.7,
                        validation_set_ratio=0.1,
                        test_set_ratio=0.2
                        )

---

# Push files to Repo

## Push generated/new files from this Session to your GitHub repo

* .gitignore

In [15]:
!cat .gitignore

core.Microsoft*
core.mongo*
core.python*
env.py
__pycache__/
*.py[cod]
node_modules/
.github/
cloudinary_python.txt
kaggle.json
inputs/mildew_dataset/cherry-leaves/test
inputs/mildew-leaves_dataset/cherry-leaves/train
inputs/mildew-leaves_dataset/cherry-leaves/validation

* Git status

In [16]:
!git status

cherry-leaves/validation/powdery_mildew/5f969553-68de-42bf-a892-0d7d6a96bb28___FREC_Pwd.M 4537.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/60c15b34-a1a5-440e-85f7-c83d848d41f9___FREC_Pwd.M 4770.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/61800495-73fd-40e8-8ddf-b45cb4670213___FREC_Pwd.M 5074.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/6269aa19-6cb1-41de-828d-e59361033b3f___FREC_Pwd.M 0565.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/62f18804-329f-41b0-bb05-3cf72f6ac5f6___FREC_Pwd.M 4838_flipLR.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/631167a9-9bde-44c5-8344-461589dd9a75___FREC_Pwd.M 4862_flipLR.JPG
	deleted:    inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/63ca2f3f-c6aa-463e-b74a-23e328637043___FREC_Pwd.M 5109.JPG
	deleted:    inputs/cherry-leaves_dataset

* Git add

In [24]:
!git add .

* Git commit

In [26]:
!git commit -am " Add data collection"

[main 6274060]  Add data collection
 1 file changed, 14 insertions(+), 4 deletions(-)


* Git Push

In [27]:
! git push

Enumerating objects: 3818, done.
Counting objects: 100% (3818/3818), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3806/3806), done.
Writing objects: 100% (3809/3809), 48.79 MiB | 13.59 MiB/s, done.
Total 3809 (delta 6), reused 1 (delta 1), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 3 local objects.
To https://github.com/HumaIlyas/mildew-detection-in-cherry-leaves
   249477d..6274060  main -> main


* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
